In [6]:
import os
from typing import List, Dict, Any
import pandas as pd


In [7]:
from langchain_core.documents import Document
from langchain_text_splitters import (
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter,
    TokenTextSplitter
)
print("All imports successful")

All imports successful


In [8]:
doc = Document(
    page_content="This is the main text content that will be embedded and searched.", 
    metadata={""
    "source": "example.txt",
    "page": 1,
    "author": "Krish Naik",
    "date_created": "2024-01-01",
    "custom_field": "any_value"
    }
)
print(f"Content: {doc.page_content}")
print(f"Metadata: {doc.metadata}")

Content: This is the main text content that will be embedded and searched.
Metadata: {'source': 'example.txt', 'page': 1, 'author': 'Krish Naik', 'date_created': '2024-01-01', 'custom_field': 'any_value'}


In [9]:
import os
os.makedirs("data/text_files", exist_ok=True)

In [10]:
sample_text = {
    "data/text_files/python_intro.txt": """
    This is the content of file one. It contains important information about LangChain. 
    LangChain is a powerful library for building applications with language models.
    
    
    It provides tools for data ingestion, parsing, and embedding.
    and more.
    
    Key features include document loaders, text splitters, and vector stores.
    Python is a versatile programming language used for various applications, including web development, data analysis, and artificial intelligence.


    Learning Python can open up many career opportunities in the tech industry.
    """,
    
}

for filepath, content in sample_text.items():
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(content)

print("Sample text files created successfully.")        

Sample text files created successfully.


## File Loader

In [13]:
from langchain_community.document_loaders import TextLoader

## Loading single text file
loader = TextLoader("data/text_files/python_intro.txt", encoding="utf-8")

documents = loader.load()
print(type(documents))
print(documents[0].metadata)


<class 'list'>
{'source': 'data/text_files/python_intro.txt'}


## Directory loader

In [20]:
from langchain_community.document_loaders import DirectoryLoader

# Loading all text files from a directory
directory_loader = DirectoryLoader(
    "data/text_files", 
    glob="**/*.txt", ## pattern to match files
    loader_cls=TextLoader, ## loader class to use
    loader_kwargs={"encoding": "utf-8"},
    show_progress=True
)

documents = directory_loader.load()
for i, doc in enumerate(documents):
    print(f"\nDocument {i+1}:")
    print(f"  Source: {doc.metadata['source']}")
    print(f"  Length of content: {len(doc.page_content)} characters")


100%|██████████| 1/1 [00:00<00:00, 1501.72it/s]


Document 1:
  Source: data/text_files/python_intro.txt
  Length of content: 570 characters
